# STATS 250 - Student Tracker
*Find the latest version at https://github.com/lpuglisi/250GradeTracking*

After changing the file name of the Canvas grade file in section 1, you run all cells by clicking 'Cell' > 'Run All'.

## Table of Contents
* [Start Here](#first)
* [Run These](#second)
* [Explore and Customize](#third)
    * [Generally, how are my sections doing?](#thirdfirst)
    * [How did my sections do on the exams?](#thirdsecond)
    * [Who is my top 10% and bottom 10% based on the Canvas current score?](#thirdthird)
    * [How is a specific student doing?](#thirdfourth)
    * [What grade is predicted for my students?](#thirdfifth)

## 1) Start Here <a class="anchor" id="first"></a>

Start by giving a filename of the Canvas grade file you would like to anaylze.

In [ ]:
#Add the filename (or file path if file is not in your current directory) of the grade file you want to import in
filename = '23_Oct_09-42_Grades-STATS_250_F17.csv'

## 2) Run These <a class="anchor" id="second"></a>

The following blocks of code do not need to be customized, they can just be run to modify the grade file from Canvas to be in a nice format. These functions will take a Canvas grade file and produce two dataframes (grades and grades2) that are easier to use for analysis. Feel free to export these files for use by running "*grades.to_csv('filename.csv')*".

In [ ]:
import pandas as pd
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.display import display, HTML

In [ ]:
#This function takes a Canvas file and outputs a dataframe with a similar format to Canvas
def MrClean(file):
    "Takes a Canvas grade file location, cleans it up, outputs a dataframe"
    df = pd.read_csv(file)
    df.drop(0, inplace=True)
    df.drop(1, inplace=True)
    df.fillna(value=0,inplace=True)
    df['ind'] = range(len(df))
    #Correctly setting all the data types
    df.ix[:,1:3] = df.ix[:,1:3].astype(int)
    df.ix[:,0:5] = df.ix[:,0:5].astype(str)
    df.ix[:,5:] = df.ix[:,5:].astype(float)
    #Renaming fields to be more user friendly
    df.columns = [(x.lower()).replace(' ', '_') for x in list(df.columns.values)]
    #Pick out column names we want to keep
    prelabnm = [sub for sub in list(df.columns) if '_prelab_'in sub]
    attdnm = [sub for sub in list(df.columns) if 'attendance_l'in sub]
    tcknm = [sub for sub in list(df.columns) if 'ticket_l'in sub]
    icklnm = [sub for sub in list(df.columns) if 'iclicker'in sub]
    lecprt = [sub for sub in list(df.columns) if 'lecpart'in sub]
    hwnm = [sub for sub in list(df.columns) if 'hw_'in sub]
    exmnm = [sub for sub in list(df.columns) if 'exam_'in sub]
    wrtlnm = [sub for sub in list(df.columns) if 'write_t'in sub]
    currscrnm = [sub for sub in list(df.columns) if 'current_score'in sub]
    fnlscrnm = [sub for sub in list(df.columns) if 'final_score'in sub]
    misc = ['ind','student','id','sis_user_id','sis_login_id','section']
    keepme = misc + currscrnm + prelabnm + attdnm + tcknm + icklnm + lecprt + hwnm + exmnm + wrtlnm  + fnlscrnm
    #Get rid of test student
    df = df[df['student']!='Student, Test']
    return df[keepme]

#Pull out the mute flag and convert to 0,1 flag
activeflg = pd.read_csv(filename)
activeflg = activeflg.ix[0,:]
activeflg = activeflg.map(pd.Series([0,1],index=['Muted',np.nan]))

In [ ]:
#These functions take the cleaned up file outputted from MrClean() and convert it to a long format
def percentscore(df):
    "This function converts the assignment points total to a percent"
    if df['assntype']=='prelab':
        scr = (df['score']/2)*100
    elif df['assntype']=='iclicker':
        scr = (df['score']/2)*100
    elif df['assntype']=='ticket':
        scr = (df['score']/2)*100
    elif df['assntype']=='homework':
        scr = (df['score']/35)*100
    elif df['assntype']=='exam':
        scr = (df['score']/75)*100
    elif df['assntype']=='lab_attendance':
        scr = (df['score']/1)*100
    elif df['assntype']=='lecture_attendance':
        scr = (df['score']/0.5)*100
    else:
        scr = df['score']
    return scr

def MrCleanAndOrganize(df):
    "This function takes the MrClean() dataframe and outputs a long format dataframe"
    dftmp = df.copy()
    dftmp.set_index(['student','id','section','ind','sis_user_id','sis_login_id'], inplace=True)
    dftmp = dftmp.stack().reset_index()
    dftmp.columns = ['student','id','section','ind','sis_user_id','sis_login_id','assignment','score']
    #Now lets create groupings for a useful assignment type flag
    cond = [(dftmp['assignment'].str.contains('_prelab_')) & (~dftmp['assignment'].str.contains('score')),
        (dftmp['assignment'].str.contains('hw_')) & (~dftmp['assignment'].str.contains('score')),
        (dftmp['assignment'].str.contains('attendance_l')) & (~dftmp['assignment'].str.contains('score')),
        (dftmp['assignment'].str.contains('ticket_l')) & (~dftmp['assignment'].str.contains('score')),
        (dftmp['assignment'].str.contains('iclicker')) & (~dftmp['assignment'].str.contains('score')),
        (dftmp['assignment'].str.contains('exam_')) & (~dftmp['assignment'].str.contains('score')),
        (dftmp['assignment'].str.contains('write_t')) & (~dftmp['assignment'].str.contains('score')),
        (dftmp['assignment'].str.contains('lecpart')) & (~dftmp['assignment'].str.contains('score')),
        (dftmp['assignment'].str.contains('current_score')),
        (dftmp['assignment'].str.contains('final_score'))]
    cats = ['prelab','homework','lab_attendance','ticket','iclicker','exam','write','lecture_attendance','current','final']
    dftmp['assntype'] = np.select(cond, cats, default='zzzz')
    dftmp['score_percent'] = dftmp.apply(percentscore,axis=1)
    #Add an mute flag for each assignment to tell if assignment is muted on Canvas
    newlab = [(x.lower()).replace(' ', '_') for x in list(activeflg.index.values)]
    act = activeflg.reset_index()
    act['index'] = newlab
    act = act.set_index(keys='index')
    act = pd.Series(act[0])
    dftmp['muteflag'] = dftmp.assignment.map(act)
    return dftmp

In [ ]:
#Load in and clean up the grades
grades = MrClean(filename)
#Create a new version of grades that is in long format
grades2 = MrCleanAndOrganize(grades)

In [ ]:
#Adding a quantile value for each student by each assignment's current score
currscrnm = [sub for sub in list(grades.columns) if 'current_score'in sub]
for i in currscrnm:
    lst = list(grades[i].values)
    grades['qnt_'+i] = [round(scp.stats.percentileofscore(lst,grd,'weak')) for grd in lst]

## 3) Explore and Customize <a class="anchor" id="third"></a>

The following sections try to answer specific questions about your labs. There are sections that explore the overall lab and sections that explore specific students. Subtitles indicate if you need to customize certain sections for students or exams you are interested in.

### 3.1) Generally, how are my sections doing? <a class="anchor" id="thirdfirst"></a>

In [ ]:
#Scatter plot of the all sections to see generally how students are doing (based on current score)
#Setting style of scatter
sns.set_context("poster", font_scale=0.8)
sns.set_style("ticks")

# Create scatterplot of dataframe
sns.lmplot(x='ind',y='current_score',data=grades, fit_reg=False, hue="section", size=10,scatter_kws={"marker": "D","s": 100})

# Define the labels
plt.title('Distribution by Lab Section')
plt.xlabel('Student')
plt.ylabel('Current Score (%)')

In [ ]:
#Generally how is each section doing
grades.pivot_table(index='section', values=currscrnm,aggfunc=np.mean)

### 3.2) How did my sections do on the exam? <a class="anchor" id="thirdsecond"></a>

#### Enter Exam

In [ ]:
#Enter 'exam_1', 'exam_2', or 'final_exam'
exam = 'exam_1'

#### Run these

In [ ]:
exmflt = grades2['assignment'].str.contains(exam)
exmsum = pd.DataFrame(grades2[exmflt].groupby('section').score_percent.describe()).round(2)
exmsum

In [ ]:
exmsum_plt = (exmsum.reset_index()).rename(columns={'level_1':'Stats'})
fiveplt = sns.factorplot(x='score_percent', y='Stats', col='section',kind='bar',  data=exmsum_plt)

#### Enter a Section Number

In [ ]:
sct = 'Section 061'

In [ ]:
sns.distplot(grades2[(exmflt) & (grades2['section'].str.contains(sct))].score_percent.map(int), rug=True)
plt.axes().set_title(sct +  ' ' + 'Distribution ('+exam+')')

### 3.3) Who is my top 10% and bottom 10% based on the Canvas current score? <a class="anchor" id="thirdthird"></a>

#### Bottom 10%

In [ ]:
#Students in the bottom 10% of current score
bott = grades[grades['qnt_current_score']<10]
bott[['student', 'id','section']+currscrnm]

#### Top 10%

In [ ]:
#Students in the top 10% of current score
top = grades[grades['qnt_current_score']>90]
top[['student', 'id','section']+currscrnm]

### 3.4) How is a specific student doing? <a class="anchor" id="thirdfourth"></a>

#### Enter an assignment and a student name (case sensitive with partial matches) or ID

In [ ]:
#Add a student name (Case sensitive) or student ID, and an assignment type you want to investigate

#Possible assignment types are:
#'current', 'prelab', 'lab_attendance', 'ticket', 'iclicker','final', 'lecture_attendance', 'homework', 'exam', 'write'

studentname = 'John'
studentid = '111111'
assignment = 'homework'

#### Run These

In [ ]:
#Look up specific student scores (checks if there are more than one student based on the name/ID)
nst = len(grades2[((grades2['student'].str.contains(studentname)) | (grades2['id']==studentid))].student.unique())
if (nst != 1):
    print("HEY! There are {} matching students based on the info you added above (try using only a student ID)".format(nst))
else:
    stdname = grades2[((grades2['student'].str.contains(studentname)) | (grades2['id']==studentid))].student.unique()[0]
    stdid = grades2[((grades2['student'].str.contains(studentname)) | (grades2['id']==studentid))].id.unique()[0]
    display(grades2[(grades2['assntype']==assignment) & (grades2['student']==stdname)])

In [ ]:
#Scatter of un-muted assignments by assignment grouping
sns.set_style("whitegrid")
sns.swarmplot(x="score_percent", y="assntype", size=10,data=grades2[(grades2['id']==stdid) & (grades2['muteflag']!=0) & (~grades2['assntype'].isin(['final','current','zzzz']))])
plt.axes().set_title('Current Grades for ' + stdname + ' (' + stdid + ')')

### 3.5) What grade is predicted for my students? <a class="anchor" id="thirdfifth"></a>

### Coming Soon...

* Calculated based on prelab, HW, exams, and attendance what the predicted grade for my student
* Try to figure out students that have a current grade higher/lower than the predicted grade
* Spot students trending up or down